Import libraries

In [1]:
import pandas as pd

"Load CSVs to transform"

The load_csv function was created to standardize the process of loading CSV files.
In the next step, we use this function to create the DataFrames that will be transformed.

In [2]:
# Define the file paths for the CSVs to be used with the load_csv function
appointments = "appointments.csv"
patients = "patients.csv"
slots = "slots.csv"

#Create load_csv funtion
def load_csv(file_path):
    try: 
        df = pd.read_csv(file_path)
        print(f"{file_path} file uploaded successfully")
        return df
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{file_path}' is empty.")
        return None
    except pd.errors.ParserError:
        print(f"Error: The file '{file_path}' contains corrupt or malformed data.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading '{file_path}': {e}")
        return None

In [14]:
#Create the dataframe that will be transformed.
df_appointments = load_csv(appointments)
df_patients = load_csv(patients)
df_slots = load_csv(slots)

appointments.csv file uploaded successfully
patients.csv file uploaded successfully
slots.csv file uploaded successfully


Appointments CSV Transformation

DataFrame preview

In [4]:
display(df_appointments.head(5))


,appointment_id,slot_id,scheduling_date,appointment_date,appointment_time,scheduling_interval,status,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id,sex,age,age_group
0,138,1,2014-12-28,2015-01-01,08:00:00,4,did not attend,NaN,NaN,NaN,NaN,NaN,8285,Male,37,35-39
1,146,23,2014-12-29,2015-01-01,13:30:00,3,did not attend,NaN,NaN,NaN,NaN,NaN,5972,Male,84,80-84
2,21,24,2014-12-17,2015-01-01,13:45:00,15,attended,13:36:45,5.2,13:37:57,13:43:09,1.2,6472,Male,77,75-79
3,233,25,2014-12-31,2015-01-01,14:00:00,1,attended,13:59:32,28.9,14:00:40,14:29:34,1.1,5376,Female,37,35-39
4,90,26,2014-12-26,2015-01-01,14:15:00,6,cancelled,NaN,NaN,NaN,NaN,NaN,8028,Male,72,70-74


Appointments CSV Transformation

Following the Exploratory Data Analysis, this CSV will be transformed for use in a transactional database management system with MySQL, so the file needs to be normalized.
The Appointments CSV contains NaN values in the columns check_in_time, appointment_duration, start_time, end_time, and waiting_time. However, in this case, we do not remove these NaN values because these fields will be completed later in the DBMS.

Transformation:

1. We remove the columns appointment_date, appointment_time, sex, age, and age_group because this information is already present in the patients and slots tables.

2. Transform the 'status' column to follow Third Normal Form (3NF) by referencing a status ID from a separate status lookup table. Then, rename the column to 'status_id'. (At the end of the transformation, we will create the status CSV that includes the 'status_id' and 'status_description'.)

3. Change data types to match MySQL format

Transformation 1

In [5]:
#Remove columns appointment_date, appointment_time, sex, age, and age_group
df_appointments = df_appointments.drop(["appointment_date", "appointment_time", "sex", "age", "age_group"], axis=1)

display(df_appointments.head(5))


,appointment_id,slot_id,scheduling_date,scheduling_interval,status,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id
0,138,1,2014-12-28,4,did not attend,NaN,NaN,NaN,NaN,NaN,8285
1,146,23,2014-12-29,3,did not attend,NaN,NaN,NaN,NaN,NaN,5972
2,21,24,2014-12-17,15,attended,13:36:45,5.2,13:37:57,13:43:09,1.2,6472
3,233,25,2014-12-31,1,attended,13:59:32,28.9,14:00:40,14:29:34,1.1,5376
4,90,26,2014-12-26,6,cancelled,NaN,NaN,NaN,NaN,NaN,8028


Transformation 2

In [6]:
#Change the values in the status column to numeric codes (did not attend = 1, attended = 2, and cancelled = 3).

df_appointments['status'] = df_appointments['status'].replace({'did not attend' : 1, 'attended' : 2, 'cancelled' : 3, 'scheduled' : 4, 'unknown' : 5})

#Rename the column status to status_id.
df_appointments.rename(columns={'status': 'status_id'}, inplace=True)

display(df_appointments.head(5))

C:\Users\nahue\AppData\Local\Temp\ipykernel_17868\825788000.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_appointments['status'] = df_appointments['status'].replace({'did not attend' : 1, 'attended' : 2, 'cancelled' : 3, 'scheduled' : 4, 'unknown' : 5})


,appointment_id,slot_id,scheduling_date,scheduling_interval,status_id,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id
0,138,1,2014-12-28,4,1,NaN,NaN,NaN,NaN,NaN,8285
1,146,23,2014-12-29,3,1,NaN,NaN,NaN,NaN,NaN,5972
2,21,24,2014-12-17,15,2,13:36:45,5.2,13:37:57,13:43:09,1.2,6472
3,233,25,2014-12-31,1,2,13:59:32,28.9,14:00:40,14:29:34,1.1,5376
4,90,26,2014-12-26,6,3,NaN,NaN,NaN,NaN,NaN,8028


Transformation 3

In [7]:
#Change data type of column 'scheduling_date' from object to datetime
df_appointments['scheduling_date'] = pd.to_datetime(df_appointments['scheduling_date'])

#Change data type of column 'status' from objetc to int64
df_appointments['status_id'] = df_appointments['status_id'].astype(int)

#Change data type of columns 'check_in_time', 'start_time' and 'end_time' from object to datetime
# When we change the datatype to time, the rows with valid values change to time,
# but the rows with NaN values change to NaT with object datatype,
# so the column keeps having object datatype because it contains two different types.
df_appointments['check_in_time'] = pd.to_datetime(df_appointments['check_in_time'], format='%H:%M:%S', errors='coerce').dt.time
df_appointments['start_time'] = pd.to_datetime(df_appointments['start_time'], format='%H:%M:%S', errors='coerce').dt.time
df_appointments['end_time'] = pd.to_datetime(df_appointments['end_time'], format='%H:%M:%S', errors='coerce').dt.time

df_appointments.dtypes

appointment_id                   int64
slot_id                          int64
scheduling_date         datetime64[ns]
scheduling_interval              int64
status_id                        int64
check_in_time                   object
appointment_duration           float64
start_time                      object
end_time                        object
waiting_time                   float64
patient_id                       int64
dtype: object

Patients CSV tranfomation

DataFrame preview

In [8]:
display(df_patients.head(5))

,patient_id,name,sex,dob,insurance
0,1,Allison Hill,Female,1946-12-30,Mediflora Nexus
1,2,Nancy Rhodes,Female,1969-02-21,BioCrest Harmony
2,3,Angie Henderson,Female,1952-01-09,BioCrest Harmony
3,4,Colleen Wagner,Female,1981-01-28,BioCrest Harmony
4,5,Christina Santos,Female,1989-05-19,CurativeWhale


On this dataframe, we only need to change the data type of the 'dob' column to match the MySQL data type.

In [9]:
#Change data type of column 'dob' from object to datetime
df_patients['dob'] = pd.to_datetime(df_patients['dob'])

df_patients.dtypes

patient_id             int64
name                  object
sex                   object
dob           datetime64[ns]
insurance             object
dtype: object

Slots CSV transformation

DataFrame preview

In [15]:
display(df_slots.head(5))

,slot_id,appointment_date,appointment_time,is_available
0,1,2015-01-01,08:00:00,False
1,2,2015-01-01,08:15:00,False
2,3,2015-01-01,08:30:00,False
3,4,2015-01-01,08:45:00,False
4,5,2015-01-01,09:00:00,False


On this dataframe, we need to change the data type of column 'appointment_date' and 'appointment_time' from object to datetime to match the MySQL data type

In [16]:
#Change data type of column 'appointment_date' from objet to datetime
df_slots['appointment_date'] = pd.to_datetime(df_slots['appointment_date'])

#Change data type of column 'appointment_time' from object to datetime
df_slots['appointment_time'] = pd.to_datetime(df_slots['appointment_time'], format='%H:%M:$S', errors='coerce').dt.time

df_slots.dtypes


slot_id                      int64
appointment_date    datetime64[ns]
appointment_time    datetime64[ns]
is_available                  bool
dtype: object

Create status CSV

We create this CSV following the normalization of the Appointments CSV (see Transformation 2 of the Appointments CSV).

In [17]:
# Declare the data dictionary containing the columns and values of the DataFrame
data = {
    'status_id': [1, 2, 3, 4, 5],
    'status_description': ['did not attend', 'attended', 'cancelled', 'scheduled', 'unknown' ]
}

#Create the dataframe status
df_status = pd.DataFrame(data)

display(df_status)

,status_id,status_description
0,1,did not attend
1,2,attended
2,3,cancelled
3,4,scheduled
4,5,unknown


Download the new and transformed CSV

In [18]:
#Download Appointment CSV transformed
df_appointments.to_csv('appointments_transformed.csv', index=False)

#Download patients CSV transformed
df_patients.to_csv('patients_transformed.csv', index=False)

#Download slots CSV transformed
df_slots.to_csv('slots_transformed.csv', index=False)

#Download status CSV (new)
df_status.to_csv('status_transformed.csv', index=False)